In [2]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import os

# Load the ampl model

In [3]:
ampl = AMPL()
modelDirectory = "models"
modelName = "two_markets.mod"
mod_path = os.path.join(modelDirectory, modelName)
dataDirectory = "data"
ampl.read(mod_path)

	models/two_markets.mod
	line 33 offset 1477
	q_DA is not defined
	models/two_markets.mod
	line 34 offset 1554
	p_DA needs 1 subscript, not 2
	models/two_markets.mod
	line 35 offset 1585
	Q_DA is not defined


# Assign set data

In [4]:
# Method to assign set data taking set name and a list as arguments
def assign_set_data(name,data):
    df = DataFrame(name)
    df.setColumn(name,data)
    ampl.setData(df,name)

In [42]:
intervals = pd.read_csv(os.path.join(dataDirectory, "intervals.csv"))
dffr = pd.read_csv(os.path.join(dataDirectory, "dffr.csv"))
da = pd.read_csv(os.path.join(dataDirectory, "da.csv"))

In [59]:
assign_set_data("INTERVALS",intervals.values[:,0])
assign_set_data("DFFR_PRICE",dffr.DFFR_PRICE.values)
assign_set_data("DA_PRICE",da.DA_PRICE.unique())

# Assign parameter data

In [66]:
df = dffr.set_index("DFFR_PRICE")
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

#df = dffr.set_index(["DA_PRICE","INTERVALS"])
#ampl_df = DataFrame.fromPandas(df)
#ampl.setData(ampl_df)


In [14]:
# Assigning a multi-indexed parameter 
# (This parameter is not used in the model yet)
idx = pd.MultiIndex.from_product([DA_PRICE,INTERVALS],
                           names = ["DA_PRICE","INTERVALS"])
p_DA = np.ones(len(idx))
df = pd.DataFrame(p_DA, index = idx, columns=["p_DA"])
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

# Solve the model

In [68]:
# Set ampl options
cplexopt = 'mipdisplay = 2 mipgap = .005'
settings = {'solver': 'cplexamp', 'cplex_options': cplexopt}
for key in settings:
    ampl.setOption(key, settings[key])

In [16]:
ampl.solve()

CPLEX 12.5.1.0: mipdisplay = 2
mipgap = .005

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 32 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
CPLEX 12.5.1.0: QP Hessian is not positive semi-definite.
0 MIP simplex iterations
0 branch-and-bound nodes
No basis.


In [1]:
# Example of getting solution
p = ampl.getVariable("q_R")
p.getValues().toPandas()

NameError: name 'ampl' is not defined

In [67]:
# Example of getting parameter values
p = ampl.getParameter("price_R")
p.getValues().toPandas()

,price_R
1.0,28.6
2.0,25.1
3.0,22.3
4.0,34.5
